In [2]:
!nvidia-smi
from tensorflow.keras import layers
import tensorflow as tf

Wed Mar 13 06:06:12 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:1E.0 Off |                    0 |
| N/A   32C    P0              26W /  70W |      2MiB / 15360MiB |      5%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

2024-03-13 06:06:13.390456: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-13 06:06:13.539139: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:460] Initializing the SageMaker Profiler.
2024-03-13 06:06:13.539261: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:105] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2024-03-13 06:06:13.539488: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-03-1

In [3]:
from helper_functions import unzip_data,create_tensorboard_callback,plot_loss_curves,compare_historys

In [4]:
#!wget https://storage.googleapis.com/ztm_tf_course/nlp_getting_started.zip

In [5]:
#unzip_data('nlp_getting_started.zip')

In [6]:
import pandas as pd
train_df = pd.read_csv("train.csv")
train_df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [7]:
train_df_shuffled = train_df.sample(frac=1, random_state = 42)

In [8]:
train_df_shuffled.head()

,id,keyword,location,text,target
2644,3796,destruction,NaN,So you have a new weapon that can cause un-ima...,1
2227,3185,deluge,NaN,The f$&amp;@ing things I do for #GISHWHES Just...,0
5448,7769,police,UK,DT @georgegalloway: RT @Galloway4Mayor: ÛÏThe...,1
132,191,aftershock,NaN,Aftershock back to school kick off was great. ...,0
6845,9810,trauma,"Montgomery County, MD",in response to trauma Children of Addicts deve...,0


In [9]:
test_df = pd.read_csv("test.csv")
test_df.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [10]:
train_df.target.value_counts()

0    4342
1    3271
Name: target, dtype: int64

In [11]:
len(train_df),len(test_df)

(7613, 3263)

In [12]:
import random
random_index = random.randint(0,len(train_df)-5)
for row in train_df_shuffled[["text","target"]][random_index:random_index+5].itertuples():
    _,text,target = row
    print(f"Target : {target}", "(real disaster)" if target > 0 else "(not real disaster)")
    print(f"Text : \n{text}\n")
    print("----\n")
    


Target : 0 (not real disaster)
Text : 
Build your own kingdom and lead your army to victory! https://youtu.

Start g this friend code: LZKTJNOX http://t.co/zZ0cEwEw64

----

Target : 0 (not real disaster)
Text : 
AD Miles 'Hurricane of Fun : The Making of Wet Hot' https://t.co/SBZwRuwuFh

----

Target : 1 (real disaster)
Text : 
Football hooligan jailed for rioting before game in Scotland was already banned from matches in England #UkNews http://t.co/q5mp2Q6Hy8

----

Target : 1 (real disaster)
Text : 
Apocalpytic Iran Heat Wave Nearly Breaks World Record http://t.co/Y8WLOcTeVC

----

Target : 1 (real disaster)
Text : 
5:15p For those who watch over me forest fire smoke has lifted 'nuff f/me t/make a run t/the pet store on the hwy (N). I'll b/home by 7p.

----



In [13]:
#007 Splitting data into training and validation sets.mp4
from sklearn.model_selection import train_test_split
train_sentences , val_sentences ,train_labels,val_labels = train_test_split(train_df_shuffled["text"].to_numpy(),
                                                                            train_df_shuffled["target"].to_numpy(),
                                                                            test_size = 0.1,
                                                                            random_state = 42)
                                                                            
                                                                            

In [14]:
len(train_sentences),len(train_labels),len(val_sentences),len(val_labels)

(6851, 6851, 762, 762)

In [15]:
#008 Converting text data to numbers using tokenisation and embeddings (overview).mp4
#009 Setting up a TensorFlow TextVectorization layer to convert text to numbers.mp4
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
#OOV  = "Out of Vocabulary"
text_vectorizer = TextVectorization(max_tokens=10000,
                                    standardize = "lower_and_strip_punctuation",
                                    split = "whitespace",
                                    ngrams = None,
                                    output_mode = "int",
                                    output_sequence_length=None,
                                    pad_to_max_tokens=True)
                                    
                                    
                                    

2024-03-13 06:06:20.464114: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 06:06:20.472550: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 06:06:20.474460: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-13 06:06:20.477059: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX512F AVX512_VNNI
To enable them in other operations, rebu

In [16]:
round(sum([len(i.split()) for i in train_sentences])/len(train_sentences))

15

In [17]:
max_vocab_length = 10000
max_length = 15

text_vectorizer = TextVectorization(max_tokens = max_vocab_length,
                                    output_mode = "int",
                                    output_sequence_length = max_length,
                                   )

In [18]:
#010 Mapping the TextVectorization layer to text data and turning it into numbers
# Fit the text vectorizer to training text
text_vectorizer.adapt(train_sentences)

In [19]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nVectorized version:")
text_vectorizer([random_sentence])

Original text:
 Earthquake : M 3.4 - 96km N of Brenas Puerto Rico: Time2015-08-05 10:34:24 UTC2015-08-05 06:34:24 -04:00 atÛ_ http://t.co/sDZrrfZhMy\n
Vectorized version:


<tf.Tensor: shape=(1, 15), dtype=int64, numpy=
array([[ 290,  772, 3211,    1,  679,    6,    1, 9487, 9084, 2651,    1,
        1242,    1,    1, 4126]])>

In [20]:
words_in_vocab = text_vectorizer.get_vocabulary()
top_5_words = words_in_vocab[:5]
bottom_5_words = words_in_vocab[-5:]
#[UNK] meand unknown token

In [21]:
len(words_in_vocab)

10000

In [22]:
#011 Creating an Embedding layer to turn tokenised text into embedding vectors
from tensorflow.keras import layers
embedding = layers.Embedding(input_dim=max_vocab_length,
                             output_dim=128,
                             input_length = max_length
                             )

In [23]:
random_sentence = random.choice(train_sentences)
print(f"Original text:\n {random_sentence}\\n\nEmbedded version:")
sample_embed = embedding(text_vectorizer([random_sentence]))
sample_embed

Original text:
 @TitorTau The Loretta Lynch one was fuckin' HI-LARIOUS to watch in realtime. It was like a train wreck of fact-checking and ombudsmanship.\n
Embedded version:


<tf.Tensor: shape=(1, 15, 128), dtype=float32, numpy=
array([[[ 0.04275957,  0.04577151,  0.04030696, ...,  0.04719367,
          0.04796653,  0.02267785],
        [-0.01810528, -0.03233446,  0.02146352, ..., -0.02864634,
         -0.04126202,  0.01741289],
        [-0.02189822,  0.00189239, -0.03043389, ..., -0.01723238,
          0.01599174,  0.0360934 ],
        ...,
        [ 0.04230091, -0.03306632, -0.04954169, ..., -0.00647932,
         -0.00248319, -0.00159659],
        [-0.03014268, -0.04838078,  0.00650382, ...,  0.0018072 ,
         -0.04419131,  0.01570432],
        [-0.0027802 ,  0.01777912,  0.01065531, ..., -0.01327743,
          0.01400194,  0.0027944 ]]], dtype=float32)>

In [24]:
#check out a single token embedding
sample_embed[0][0],sample_embed[0][0].shape,random_sentence

(<tf.Tensor: shape=(128,), dtype=float32, numpy=
 array([ 0.04275957,  0.04577151,  0.04030696, -0.04479456,  0.04757308,
        -0.04914287, -0.04022181,  0.00127598,  0.0437136 , -0.01435585,
         0.02080286,  0.03701954,  0.00317318,  0.02803612,  0.04807443,
        -0.01174543,  0.00398467, -0.00950684,  0.01188948, -0.02639636,
        -0.02833161,  0.04864627,  0.02974888,  0.01220663,  0.04670924,
         0.01415093, -0.03951336,  0.03488387,  0.00719555, -0.01459559,
        -0.04244563,  0.01943988,  0.03411445,  0.00646008,  0.01625749,
         0.0472548 ,  0.03088847, -0.04612533, -0.00681605, -0.04283381,
        -0.0183815 ,  0.0243837 ,  0.00093026,  0.00120956,  0.00375264,
         0.02878701,  0.03216357, -0.03295897, -0.01955888, -0.02496184,
        -0.00625015, -0.00114731,  0.04563365, -0.04324099, -0.00061327,
         0.00939617,  0.04202   , -0.02929084,  0.01431799, -0.03379945,
         0.01009095,  0.04461069, -0.0275233 ,  0.00550486,  0.03479629,
  

In [25]:
#012 Discussing the various modelling experiments we're going to run
#013 Model 0 Building a baseline model to try and improve upon
#Multinomial naive bais using TF_IDF formula
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline

model_0 = Pipeline([
    ("tfidf",TfidfVectorizer()),
    ("clf",MultinomialNB()),

])
model_0.fit(train_sentences,train_labels)

Pipeline(steps=[('tfidf', TfidfVectorizer()), ('clf', MultinomialNB())])

In [26]:
#Evalute our baseline model
baseline_score = model_0.score(val_sentences,val_labels)
print(f"Our baseline model achives an accuracy of : {baseline_score*100:.2f}%")

Our baseline model achives an accuracy of : 79.27%


In [27]:
baseline_preds = model_0.predict(val_sentences)
baseline_preds[:20]

array([1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1])

In [28]:
#014 Creating a function to track and evaluate our model's results
from sklearn.metrics import accuracy_score,precision_recall_fscore_support

def calculate_results(y_true,y_pred):
    model_accuracy = accuracy_score(y_true,y_pred)*100
    model_precision,model_recall,model_f1,_=precision_recall_fscore_support(y_true, y_pred,average="weighted")
    model_results = {"accuracy":model_accuracy,
                     "precision":model_precision,
                     "recall":model_recall,
                     "f1":model_f1}
    return model_results

In [29]:
baseline_results = calculate_results(y_true=val_labels,y_pred=baseline_preds)
baseline_results
#higher precision leads to less false positives
#higher recall leads to less false negetives

{'accuracy': 79.26509186351706,
 'precision': 0.8111390004213173,
 'recall': 0.7926509186351706,
 'f1': 0.7862189758049549}

In [30]:
#015 Model 1 Building, fitting and evaluating our first deep model on text data
from helper_functions import create_tensorboard_callback
from tensorflow.keras import layers
SAVE_DIR = "model_logs"
#build model with Function API
inputs = layers.Input(shape=(1,),dtype=tf.string)
x = text_vectorizer(inputs) # turn input text into numbers
x = embedding(x)
x = layers.GlobalAveragePooling1D()(x) #layers.Reshape((1920,))(x) #.
outputs = layers.Dense(1,activation="sigmoid")(x)
model_1 = tf.keras.Model(inputs,outputs,name = "model_1_dense")
model_1.summary()



Model: "model_1_dense"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization_1 (TextV  (None, 15)               0         
 ectorization)                                                   
                                                                 
 embedding (Embedding)       (None, 15, 128)           1280000   
                                                                 
 global_average_pooling1d (G  (None, 128)              0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 1,280,129
Trainable params: 1,280,129
N

In [ ]:
model_1.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=["accuracy"])

model_1_history = model_1.fit(x=train_sentences,
                              y=train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(dir_name=SAVE_DIR,
                                                                     experiment_name="model_1_dense")])

Saving TensorBoard log files to: model_logs/model_1_dense/20240313-060624
Epoch 1/5


2024-03-13 06:06:25.618702: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x560c9213f720 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-03-13 06:06:25.618749: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-03-13 06:06:25.625444: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-03-13 06:06:25.838734: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


215/215 [==============================] - 13s 53ms/step - loss: 0.6113 - accuracy: 0.6856 - val_loss: 0.5373 - val_accuracy: 0.7559
Epoch 2/5
215/215 [==============================] - 3s 16ms/step - loss: 0.4426 - accuracy: 0.8165 - val_loss: 0.4705 - val_accuracy: 0.7835
Epoch 3/5
215/215 [==============================] - 2s 9ms/step - loss: 0.3479 - accuracy: 0.8610 - val_loss: 0.4585 - val_accuracy: 0.7913
Epoch 4/5
215/215 [==============================] - 2s 8ms/step - loss: 0.2850 - accuracy: 0.8910 - val_loss: 0.4654 - val_accuracy: 0.7913
Epoch 5/5
195/215 [==========================>...] - ETA: 0s - loss: 0.2372 - accuracy: 0.9112

In [ ]:
model_1.evaluate(val_sentences,val_labels)

In [ ]:
model_1_pred_probs = model_1.predict(val_sentences)
model_1_pred_probs[0]

In [ ]:
model_1_preds = tf.squeeze(tf.round(model_1_pred_probs))

In [ ]:
model_1_preds

In [ ]:
#calculate model 1 results
model_1_results = calculate_results(y_true=val_labels,
                                    y_pred=model_1_preds)
model_1_results

In [ ]:
baseline_results

In [ ]:
#016 Visualising our model's learned word embeddings with TensorFlow's projector tool
words_in_vocab = text_vectorizer.get_vocabulary()

In [ ]:
len(words_in_vocab),words_in_vocab[:10]

In [ ]:
model_1.summary()

In [ ]:
embed_weights = model_1.get_layer("embedding").get_weights()[0]
embed_weights.shape
#https://projector.tensorflow.org/
#https://www.tensorflow.org/text/guide/word_embeddings#retrieve_the_trained_word_embeddings_and_save_them_to_disk

In [ ]:
import io
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(words_in_vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = embed_weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

In [ ]:
#017 High-level overview of Recurrent Neural Networks (RNNs) + where to learn more
#018 Model 2 Building, fitting and evaluating our first TensorFlow RNN model (LSTM)
#MOdel 2 LSTM
from tensorflow.keras import layers
inputs = layers.Input(shape = (1,),dtype = "string")
x = text_vectorizer(inputs)
x = embedding(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
# x = layers.LSTM(units=64,return_sequences=True)(x)
x = layers.LSTM(64)(x)
# x = layers.Dense(64,activation="relu")(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_2 = tf.keras.Model(inputs,outputs,name="model_2_LSTM")


In [ ]:
model_2.summary()

In [ ]:
model_2.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_2_history = model_2.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_2_LSTM")])

In [ ]:
#make prediction with LSTM Model
model_2_pred_probs = model_2.predict(val_sentences)
model_2_pred_probs[:10]

In [ ]:
model_2_pred=tf.squeeze(tf.round(model_2_pred_probs))

In [ ]:
model_2_results = calculate_results(y_true=val_labels,
                                    y_pred=model_2_pred)

In [ ]:
model_2_results

In [ ]:
baseline_results

In [ ]:
#019 Model 3 Building, fitting and evaluating a GRU-cell powered RNN
#Gated Recurrent Unit
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,),dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.GRU(64)(x)
# x= layers.LSTM(64,return_sequences=True)(x)
# x = layers.GRU(64)(x)
# x= layers.GlobalAveragePooling1D()(x)
x = layers.Dense(64,activation='relu')(x)
outputs = layers.Dense(1,activation='sigmoid')(x)
model_3 = tf.keras.Model(inputs,outputs,name = "model_3_GRU")

In [ ]:
model_3.summary()

In [ ]:
model_3.compile(loss="binary_crossentropy",
                optimizer=tf.keras.optimizers.Adam(),
                metrics=['accuracy'])
model_3_history = model_3.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,
                                                                       "model_3_GRU")])

In [ ]:
model_3_pred_probs = model_3.predict(val_sentences)
model_3_pred=tf.squeeze(tf.round(model_3_pred_probs))
model_3_results = calculate_results(y_true=val_labels,
                                    y_pred=model_3_pred)
model_3_results

In [ ]:
model_2_results

In [ ]:
#020 Model 4 Building, fitting and evaluating a bidirectional RNN model
from tensorflow.keras import layers
inputs = layers.Input(shape=(1,), dtype=tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x= layers.Bidirectional(layers.LSTM(64,return_sequences=True))(x)
x = layers.Bidirectional(layers.GRU(64))(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_4 = tf.keras.Model(inputs,outputs,name="model_4_bidirectional")

In [ ]:
model_4.summary()

In [ ]:
model_4.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])
model_4_history = model_4.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_4_bidirectional")])
            

In [ ]:
model_4_pred_probs = model_4.predict(val_sentences)

In [ ]:
model_4_preds = tf.squeeze(tf.round(model_4_pred_probs))

In [ ]:
model_4_results = calculate_results(y_true=val_labels,y_pred = model_4_preds)

In [ ]:
model_4_results

In [ ]:
model_3_results

In [ ]:
#021 Discussing the intuition behind Conv1D neural networks for text and sequences
embedding_test = embedding(text_vectorizer(["this is a test sentence"]))
conv_1d = layers.Conv1D(filters = 64,
                        kernel_size = 5,
                        activation="relu",
                        padding = "same")
conv_1d_output = conv_1d(embedding_test)
max_pool = layers.GlobalMaxPool1D()
max_pool_output = max_pool(conv_1d_output)

embedding_test.shape ,conv_1d_output.shape , max_pool_output.shape

In [ ]:
embedding_test

In [ ]:
max_pool_output

In [ ]:
#022 Model 5 Building, fitting and evaluating a 1D CNN for text
from tensorflow.keras import layers
input = layers.Input(shape=(1,), dtype = tf.string)
x = text_vectorizer(inputs)
x = embedding(x)
x = layers.Conv1D(filters = 64,kernel_size=5,strides = 1,activation = "relu",padding="valid")(x)
x = layers.GlobalMaxPool1D()(x)
outputs = layers.Dense(1,activation="sigmoid")(x)
model_5 = tf.keras.Model(inputs,outputs,name="model_5_Conv1D")

model_5.compile(loss="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics = ['accuracy'])

In [ ]:
model_5.summary()

In [ ]:
model_5_history = model_5.fit(train_sentences,
                              train_labels,
                              epochs = 5,
                              validation_data=(val_sentences,val_labels),
                             callbacks=[create_tensorboard_callback(SAVE_DIR,"model_5_Conv1D")])

In [ ]:
model_5_pred_probs = model_5.predict(val_sentences)

In [ ]:
model_5_preds = tf.squeeze(tf.round(model_5_pred_probs))

In [ ]:
model_5_results = calculate_results(y_true = val_labels,
                                    y_pred = model_5_preds,)
model_5_results

In [ ]:
baseline_results

In [ ]:
#023 Using TensorFlow Hub for pretrained word embeddings (transfer learning for NLP)
#Universal sentence encoder
sample_sentence = "There's a flood in my street!"
import tensorflow_hub as hub
import tensorflow as tf

embed = hub.load("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2")
with tf.device('/CPU:0'):
    embed_samples = embed([sample_sentence, "When you call the universal sentence encoder on a sentence, it turns it into numbers."])
print(embed_samples)


In [ ]:
#024 Model 6 Building, training and evaluating a transfer learning model for NLP
# create a keras layer using the USE pretrained layer from tensorflow hub
with tf.device('/CPU:0'):
    sentence_encoder_layer = hub.KerasLayer("https://www.kaggle.com/models/google/universal-sentence-encoder/frameworks/TensorFlow2/variations/universal-sentence-encoder/versions/2",input_shape = [],dtype = tf.string,trainable=False,name = "USE")

In [ ]:
# create model using the sequential API
model_6 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name = "model_6_USE")

model_6.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.Adam(),metrics=["accuracy"])


In [ ]:
model_6.summary()

In [ ]:
#Train a classifier on top of USE pretrained embeddings
# with tf.device('/CPU:0'):
model_6_history = model_6.fit(train_sentences,
                              train_labels,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder_USE")])

In [ ]:
with tf.device('/CPU:0'):
    model_6_pred_probs = model_6.predict(val_sentences)
    model_6_preds = tf.squeeze(tf.round(model_6_pred_probs))
    model_6_results = calculate_results(y_true=val_labels,y_pred=model_6_preds)


In [ ]:
model_6_results

In [ ]:
baseline_results

In [ ]:
#025 Preparing subsets of data for model 7 (same as model 6 but 10% of data)
# Model7 : Tf Hub pretrained USE but with 10% of training data
# Transfer Leraning helps when we do not have large dataset
#Below code causes data leakage
# train_10_percent = train_df_shuffled[["text","target"]].sample(frac = 0.1,random_state=42)
# train_sentences_10_percent = train_10_percent["text"].to_list()
# train_labels_10_percent = train_10_percent["target"].to_list()

# Fixing data leakage
train_10_percent_split = int(0.1*len(train_sentences))
train_sentences_10_percent = train_sentences[:train_10_percent_split]
train_labels_10_percent = train_labels[:train_10_percent_split]


In [ ]:
# train_labels_10_percent["target"].value_counts()

In [ ]:
#026 Model 7 Building, training and evaluating a transfer learning model on 10% data
# model_7 = tf.keras.models.clone_model(model_6)
model_7 = tf.keras.Sequential([
    sentence_encoder_layer,
    layers.Dense(64,activation="relu"),
    layers.Dense(1,activation="sigmoid")
],name = "model_7_USE")
model_7.compile(loss ="binary_crossentropy",
                optimizer = tf.keras.optimizers.Adam(),
                metrics=["accuracy"])
model_7.summary()

In [ ]:
model_7_history = model_7.fit(train_sentences_10_percent,
                              train_labels_10_percent,
                              epochs=5,
                              validation_data=(val_sentences,val_labels),
                              callbacks=[create_tensorboard_callback(SAVE_DIR,"tf_hub_sentence_encoder_USE_10_percent_correct_split")])

In [ ]:
#make Predictions with 10 percent model
model_7_pred_probs = model_7.predict(val_sentences)
model_7_preds = tf.squeeze(tf.round(model_7_pred_probs))
model_7_results = calculate_results(y_true=val_labels,
                                    y_pred= model_7_preds)

In [ ]:
model_7_results

In [ ]:
#027 Fixing our data leakage issue with model 7 and retraining it.mp4
#028 Comparing all our modelling experiments evaluation metrics
# Combine model results in to dataframe
all_model_results = pd.DataFrame({"0_baseline":baseline_results,
                                 "1_simple_dense":model_1_results,
                                 "2_lstm":model_2_results,
                                 "3_gru":model_3_results,
                                 "4_bidirectional":model_4_results,
                                 "5_conv1d":model_5_results,
                                 "6_tf_hub_use_encoder":model_6_results,
                                 "7_tf_hub_use_encoder_10_percent":model_7_results,})

all_model_results=all_model_results.transpose()

In [ ]:
all_model_results["accuracy"]=all_model_results["accuracy"]/100
all_model_results

In [ ]:
all_model_results.plot(kind = "bar",figsize=(10,7)).legend(bbox_to_anchor=(1.0,1.0))

In [ ]:
# Sord Model results by F1 Score
all_model_results.sort_values("f1",ascending=False)["f1"].plot(kind = "bar",figsize=(10,7))

In [ ]:
#029 Uploading our model's training logs to TensorBoard and comparing them
#https://d-mazkegzhipax.studio.eu-central-1.sagemaker.aws/jupyter/default/proxy/6006/
#Cd in root and then : tensorboard --logdir tensorflow/NLPFundamentals/model_logs
#030 Saving and loading in a trained NLP model with TensorFlow
# Saving model in hdf5 and other in savedmodel format
# model_6.save("model_6.h5")
# loaded_model_6  =tf.keras.models.load_model("model_6.h5",custom_objects={"KerasLayer":hub.KerasLayer})

In [ ]:
# loaded_model_6.evaluate(val_sentences,val_labels)

In [ ]:
#model_6.save("model_6_savedModel_fromat")

In [ ]:
#031 Downloading a pretrained model and preparing data to investigate predictions
# Model Driven Data Exploration
model_6.evaluate(val_sentences,val_labels)

In [ ]:
model_6_preds

In [ ]:
val_df = pd.DataFrame({"text":val_sentences,
                     "target":val_labels,
                     "pred":model_6_preds,
                     "pred_prob":tf.squeeze(model_6_pred_probs)})
val_df.head()

In [ ]:
most_wrong = val_df[val_df["target"]  != val_df["pred"]].sort_values("pred_prob",ascending = False)

In [ ]:
most_wrong

In [ ]:
#032 Visualising our model's most wrong predictions
for row in most_wrong[:10].itertuples():
    _,text,target,pred,pred_prob = row
    print(f"Traget : {target}, Pred: {pred}, Prob: {pred_prob}")
    print(f"Text:\n{text}\n")
    print("--------\n")

In [ ]:
#033 Making and visualising predictions on the test dataset
#034 Understanding the concept of the speedscore tradeoff
##The Speed/Score Tradeoff